# Übung 1c)

Laden der Bibliotheken

In [139]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, RobustScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import FeatureUnion

Laden des Datensatzes

In [140]:
data = pd.read_csv("../../data/bikesharing_simple.csv")


In [141]:
data.head()

,Unnamed: 0,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,casual,registered,cnt,windspeed
0,0,1,2011-01-01,spring,0,1,0,0,Saturday,No,Clear,0.24,0.2879,0.81,3,13,16,0.0
1,1,2,2011-01-01,spring,0,1,1,0,Saturday,No,Clear,0.22,0.2727,0.80,8,32,40,0.0
2,2,3,2011-01-01,spring,0,1,2,0,Saturday,No,Clear,0.22,0.2727,0.80,5,27,32,0.0
3,3,4,2011-01-01,spring,0,1,3,0,Saturday,No,Clear,0.24,0.2879,0.75,3,10,13,0.0
4,4,5,2011-01-01,spring,0,1,4,0,Saturday,No,Clear,0.24,0.2879,0.75,0,1,1,0.0


In [142]:
#Drop unnötige Spalten
data = data.drop(columns=['dteday', "Unnamed: 0"])

Prüfen der Daten auf NAN's

In [143]:
data.isna().any()

instant       False
season        False
yr            False
mnth          False
hr            False
holiday       False
weekday       False
workingday    False
weathersit    False
temp          False
atemp         False
hum           False
casual        False
registered    False
cnt           False
windspeed      True
dtype: bool

In [144]:
data.windspeed.mean()

0.19049675923173073

In [145]:
#Ersetzen Nullwerte in Spalte Windspeed mit dem Mean
data["windspeed"].fillna((data.windspeed.mean()), inplace=True)


In [146]:
data.isna().any()

instant       False
season        False
yr            False
mnth          False
hr            False
holiday       False
weekday       False
workingday    False
weathersit    False
temp          False
atemp         False
hum           False
casual        False
registered    False
cnt           False
windspeed     False
dtype: bool

In [84]:
categorical_columns = data[["season", "weekday", "workingday", "weathersit"]]
numeric_columns = data[["cnt", "registered", "casual", "windspeed", "hum", "atemp", "temp", "instant"]]
date_columns = data[["yr", "mnth", "holiday", "hr"]]

Skalieren der numerischen Daten

Label Encoding der kategorischen Variablen

In [149]:
class MultiColumnLabelEncoder:
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [154]:
data = MultiColumnLabelEncoder(columns = ["season", "weekday", "workingday", "weathersit"]).fit_transform(data)

In [156]:
data.head()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,casual,registered,cnt,windspeed
0,1,1,0,1,0,0,2,0,0,0.24,0.2879,0.81,3,13,16,0.0
1,2,1,0,1,1,0,2,0,0,0.22,0.2727,0.80,8,32,40,0.0
2,3,1,0,1,2,0,2,0,0,0.22,0.2727,0.80,5,27,32,0.0
3,4,1,0,1,3,0,2,0,0,0.24,0.2879,0.75,3,10,13,0.0
4,5,1,0,1,4,0,2,0,0,0.24,0.2879,0.75,0,1,1,0.0


Robust Scalerg auf numerischen Daten anwenden

In [165]:
scaler = RobustScaler()
#scaler.fit(data[["cnt", "registered", "casual", "windspeed", "hum", "atemp", "temp", "instant"]]
data[["cnt", "registered", "casual", "windspeed", "hum", "atemp", "temp", "instant"]]= scaler.fit_transform(data[["cnt", "registered", "casual", "windspeed", "hum", "atemp", "temp", "instant"]])


In [166]:
data.head()

,instant,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,casual,registered,cnt,windspeed
0,-1.000000,1,0,1,0,0,2,0,0,-0.8125,-0.683918,0.600000,-0.318182,-0.548387,-0.522822,-1.276788
1,-0.999885,1,0,1,1,0,2,0,0,-0.8750,-0.736714,0.566667,-0.204545,-0.446237,-0.423237,-1.276788
2,-0.999770,1,0,1,2,0,2,0,0,-0.8750,-0.736714,0.566667,-0.272727,-0.473118,-0.456432,-1.276788
3,-0.999655,1,0,1,3,0,2,0,0,-0.8125,-0.683918,0.400000,-0.318182,-0.564516,-0.535270,-1.276788
4,-0.999540,1,0,1,4,0,2,0,0,-0.8125,-0.683918,0.400000,-0.386364,-0.612903,-0.585062,-1.276788


Input und Output Variablen festlegen

In [167]:
input_features = [
    'instant',
    'season',
    'yr',
    'mnth',
    'hr',
    'holiday',
    'weekday',
    'workingday',
    'weathersit',
    "temp",
    "atemp",
    "hum",
    "windspeed",
    "casual",
    "registered"
]

output_features = [
    'cnt'
]

X_train, X_test, y_train, y_test = train_test_split(
    data[input_features],
    data[output_features]
)